In [1]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

In [2]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template(
  """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
  """
)


class Classification(BaseModel):
  sentiment: str = Field(description="The sentiment of the text")
  aggressiveness: int = Field(
    description="How aggressive the text is on a scale from 1 to 10"
  )
  language: str = Field(description="The language the text is written in")
  eng_ver: str = Field(description="The English version of the paragraph")
  
structured_llm = llm.with_structured_output(Classification)

In [7]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
response = structured_llm.invoke(prompt)

In [8]:
response

Classification(sentiment='Positive', aggressiveness=1, language='Spanish', eng_ver="I'm incredibly happy to have met you! I think we'll be very good friends!")

In [9]:
response.model_dump()

{'sentiment': 'Positive',
 'aggressiveness': 1,
 'language': 'Spanish',
 'eng_ver': "I'm incredibly happy to have met you! I think we'll be very good friends!"}